# 다중 실내기 협조제어를 위한 도달시간 예측 모델 구축 보고서

## 1. 프로젝트 개요 및 연구 목적

본 프로젝트는 **다중 실내기가 동시에 운전되는 공간에서 실내 온도를 빠르고 균일하게 낮추면서 에너지 사용을 최소화하는 협조제어 전략**을 개발하는 것을 목표로 한다.  
특히 하나의 공간에 여러 대의 실내기가 설치된 경우, 개별 실내기의 제어 결과가 서로 영향을 주기 때문에 단일 실내기 기준의 제어 전략으로는 **온도 불균형**, **불필요한 에너지 소모**, **제어 응답의 비효율성**이 발생할 수 있다.

이러한 문제를 해결하기 위해 본 과제에서는 제어기를 직접 설계하기에 앞서,  
**현재 운전 조건과 환경이 주어졌을 때 각 실내기가 목표 온도에 도달하는 데 필요한 시간**을 예측하는 모델을 구축하였다.  
이 도달시간 예측 모델은 이후 협조제어 알고리즘에서 판단 기준(비용 또는 가치 함수)으로 활용될 수 있는 핵심 구성 요소이다.

---

## 2. 도달시간 예측을 제어 핵심 지표로 선택한 이유

본 프로젝트에서 예측 대상으로 **온도 변화량**이나 **미래 온도 자체**가 아닌  
**목표 온도 도달시간**을 선택한 이유는 다음과 같다.

도달시간은 특정 운전 상태를 유지했을 때 얻어지는 **제어 결과의 요약 지표**이다.  
즉, 현재 상태에서 특정 입력을 유지한다고 가정하면,

- 도달시간이 짧을수록 제어 성능이 우수하고
- 도달시간이 길수록 제어 성능이 열악하다

라고 해석할 수 있다.

이러한 특성으로 인해 도달시간은 다음과 같은 의미를 갖는다.

- 제어 행동의 **효과를 하나의 스칼라 값으로 압축**
- 강화학습 또는 최적화 관점에서의 **행동 가치(Value)** 와 유사한 해석 가능
- 다중 실내기 환경에서 **가장 늦게 도달하는 실내기를 기준으로 협조제어 목표 설정 가능**

따라서 본 프로젝트에서는  
**“제어 결과를 직접 학습시키는 대신, 제어 결과를 대표하는 도달시간을 예측하도록 모델을 설계”**하였다.

---

## 3. 도달시간 예측 모델의 입력·출력 구성

도달시간 예측 모델은 **외생 변수**와 **실내기별 상태 및 입력**을 동시에 고려하여 각 실내기의 도달시간을 출력한다.

### 3.1 입력 변수 구성

입력 변수는 물리적 의미에 따라 다음 세 그룹으로 구성된다.

#### (1) 외생 변수 (환경 및 설비 조건)
외생 변수는 제어 대상이 아니지만 냉방 성능과 도달시간에 큰 영향을 미치는 배경 조건이다.

| 의미 | 변수명 |
|---|---|
| 외기 온도 | `Tod` |
| 실내 습도 지표 | `Hid` |
| 실외기 목표 고압 | `Tgt_Hi` |
| 실외기 목표 저압 | `Tgt_Low` |

#### (2) 실내기 상태 (현재 냉방 진행 정도)
실내기 상태는 목표까지 남은 냉방량을 나타낸다.

| 의미 | 변수명 |
|---|---|
| 실내 대표 온도 | `Tbdy` |
| 목표 대비 온도 차이 | `Tbdy - Tgoal` |

#### (3) 실내기 입력 및 냉각원 상태
실제 냉각 강도와 냉각원 상태를 간접적으로 표현하는 변수이다.

| 의미 | 변수명 |
|---|---|
| 풍량  | `Frun`(단순 풍량 설정 단계) -> 부피유량 변환 후 'flow' |
| 냉각원 대표 온도 | `Tc_ema` (냉매 파이프 인, 아웃온도를 평균 낸 후 지수이동 평균으로 업데이트)|

### 3.2 출력 변수

| 의미 | 설명 |
|---|---|
| 도달시간 | 목표 온도에 도달하기까지의 예상 시간 (분 단위) |

모델은 한 번의 추론에서 **여러 실내기의 도달시간을 동시에 출력**하도록 설계되었다.

---

## 4. 전처리 설계 철학: 도달시간을 정의 가능하게 만드는 조건

도달시간 예측이 의미를 갖기 위해서는  
**현재 상태 이후에도 동일한 운전 조건이 유지된다는 가정**이 성립해야 한다.

### 4.1 운전 조건 유지 구간만 사용한 이유

운전 중간에 설정 값이나 운전 상태가 변경되면,  
해당 시점에서 정의한 도달시간은 더 이상 “현재 상태의 결과”가 아니다.

이를 방지하기 위해 전처리 단계에서 다음 조건을 만족하는 구간만 학습 데이터로 사용하였다.

- 운전 ON/OFF 상태가 유지되는 구간
- 송풍량 및 설정 조건이 일정한 구간
- 냉방 활성 상태 패턴이 유지되는 구간

이러한 전처리는 데이터 수를 줄이기 위한 것이 아니라,  
**도달시간이라는 물리적·제어적 개념을 명확히 정의하기 위한 필수 조건**이다.

---

## 5. 손실 함수 설계와 냉각 법칙 기반 논리

본 모델은 단순 회귀 모델이 아니라,  
**냉각 동역학의 구조를 학습 과정에 반영**하도록 손실 함수를 설계하였다.

### 5.1 냉각 법칙 기반 온도 모델

실내 온도 변화는 다음과 같은 1차 냉각 모델로 근사할 수 있다.

$$
T(t) = T_{\mathrm{eq}} + \left(T_0 - T_{\mathrm{eq}}\right)
\exp\left(-\frac{t}{\tau}\right)
$$

여기서  
- $T_0$는 현재 온도  
- $T_{\mathrm{eq}}$는 충분히 냉방이 지속되었을 때의 평형 온도  
- $\tau$는 냉각 시간상수이다.

도달시간은 이 모델에서 특정 임계 온도에 도달하는 데 필요한 시간으로 해석된다.

---

### 5.2 도달시간 회귀 손실

도달시간 예측의 기본 목표는 실제 도달시간과의 오차를 최소화하는 것이다.

$$
\mathcal{L}_{\mathrm{reg}}
=
\frac{1}{N}\sum_{i}
\left(\hat{\tau}_i - \tau_i\right)^2
$$

이를 통해 모델은 각 실내기의 냉각 속도를 직접 학습한다.

---

### 5.3 트래젝토리 손실: 중간 냉각 과정 반영

도달시간만 맞출 경우, 중간 온도 변화가 비현실적인 형태로 예측될 수 있다.  
이를 보완하기 위해 예측된 도달시간으로부터 미래 시점의 온도를 복원하고 실제 관측값과 비교한다.

$$
\mathcal{L}_{\mathrm{traj}}
=
\sum_{k}
\left(
\hat{T}(t_k) - T^{\mathrm{obs}}(t_k)
\right)^2
$$

이 손실은 모델이 **냉각 곡선의 형태 자체**를 학습하도록 유도한다.

---

### 5.4 물리 정합 손실: 냉각 방향성 일관성

냉각 과정에서 온도 변화율은 유량과 냉각원 온도 차이에 의해 결정된다.

$$
\frac{dT}{dt}
\propto
-\dot{V}(T - T_c)
$$

이를 기반으로 온도 변화율과 냉각 구동 항의 방향 정합성을 손실로 구성한다.

$$
\mathcal{L}_{\mathrm{phys}}
=
1 - \cos
\left(
\frac{dT}{dt},
-\dot{V}(T - T_c)
\right)
$$

이 항은 모델이 물리적으로 일관된 냉각 패턴을 학습하도록 돕는다.

---

## 6. 협조제어 목적을 반영한 학습 기준

본 프로젝트의 목표는 특정 실내기 하나의 성능이 아니라,  
**모든 실내기의 온도가 고르게 떨어지도록 만드는 것**이다.

이를 반영하여 모델 선택 시  
실내기별 오차 중 **가장 큰 오차를 최소화**하는 기준을 사용하였다.

이는 협조제어에서 중요한  
“최악 실내기가 전체 성능을 저해하지 않도록 하는 전략”을 학습 과정에 직접 반영한 것이다.

---

## 7. 학습 결과 제시 영역

아래에는 학습 과정에서의 손실 변화를 나타내는 그래프를 삽입한다.

**[그림 1] 학습 과정에서의 손실 변화**

- 가로축: Epoch  
- 세로축: Loss 또는 RMSE  
- 포함 항목:  
  - 도달시간 회귀 손실  
  - 트래젝토리 손실  
  - 물리 정합 손실  

![](images/learning%20loss.png) 

---

## 8. 결론

본 프로젝트에서는 다중 실내기 환경에서의 협조제어를 목표로,  
**제어 결과를 직접 학습하는 대신 도달시간을 예측하는 모델**을 구축하였다.

운전 조건이 유지되는 구간만을 사용한 전처리,  
냉각 법칙을 반영한 손실 함수 설계,  
그리고 최악 실내기를 기준으로 한 학습 기준을 통해  
도달시간 예측 모델이 향후 협조제어 의사결정의 핵심 지표로 활용될 수 있음을 보였다.

이 모델은 이후 최적화 기반 제어, 모델 예측 제어, 강화학습 기반 협조제어로 확장 가능한 기반 예측기로 활용될 수 있다.
